<a href="https://colab.research.google.com/github/leemordechai/FLAME/blob/master/FLAME_PAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import requests, json

from google.colab import files

import matplotlib.pyplot as plt
from pylab import savefig
%matplotlib inline

import io
import ast

In [ ]:
#@title Conversion from British coordinates to Lat/Long
# Converts eastings and northings (British national grid coordinates) to Lat/Long
# Original code author: Hannah Fry; see code/comments here: 
# http://www.hannahfry.co.uk/blog/2012/02/01/converting-british-national-grid-to-latitude-and-longitude-ii

from math import sqrt, pi, sin, cos, tan, atan2 as arctan2
import csv

def OSGB36toWGS84(E, N):
    # E, N are the British national grid coordinates - eastings and northings
    # The Airy 180 semi-major and semi-minor axes used for OSGB36 (m)
    a, b = 6377563.396, 6356256.909
    F0 = 0.9996012717  # scale factor on the central meridian
    lat0 = 49 * pi / 180  # Latitude of true origin (radians)
    # Longtitude of true origin and central meridian (radians)
    lon0 = -2 * pi / 180
    N0, E0 = -100000, 400000  # Northing & easting of true origin (m)
    e2 = 1 - (b * b) / (a * a)  # eccentricity squared
    n = (a - b) / (a + b)

    # Initialise the iterative variables
    lat, M = lat0, 0

    while N - N0 - M >= 0.00001:  # Accurate to 0.01mm
        lat = (N - N0 - M) / (a * F0) + lat
        M1 = (1 + n + (5. / 4) * n**2 + (5. / 4) * n**3) * (lat - lat0)
        M2 = (3 * n + 3 * n**2 + (21. / 8) * n**3) * \
            sin(lat - lat0) * cos(lat + lat0)
        M3 = ((15. / 8) * n**2 + (15. / 8) * n**3) * \
            sin(2 * (lat - lat0)) * cos(2 * (lat + lat0))
        M4 = (35. / 24) * n**3 * sin(3 * (lat - lat0)) * cos(3 * (lat + lat0))
        # meridional arc
        M = b * F0 * (M1 - M2 + M3 - M4)

    # transverse radius of curvature
    nu = a * F0 / sqrt(1 - e2 * sin(lat)**2)

    # meridional radius of curvature
    rho = a * F0 * (1 - e2) * (1 - e2 * sin(lat)**2)**(-1.5)
    eta2 = nu / rho - 1

    secLat = 1. / cos(lat)
    VII = tan(lat) / (2 * rho * nu)
    VIII = tan(lat) / (24 * rho * nu**3) * \
        (5 + 3 * tan(lat)**2 + eta2 - 9 * tan(lat)**2 * eta2)
    IX = tan(lat) / (720 * rho * nu**5) * \
        (61 + 90 * tan(lat)**2 + 45 * tan(lat)**4)
    X = secLat / nu
    XI = secLat / (6 * nu**3) * (nu / rho + 2 * tan(lat)**2)
    XII = secLat / (120 * nu**5) * (5 + 28 * tan(lat)**2 + 24 * tan(lat)**4)
    XIIA = secLat / (5040 * nu**7) * (61 + 662 * tan(lat) **
                                      2 + 1320 * tan(lat)**4 + 720 * tan(lat)**6)
    dE = E - E0

    # These are on the wrong ellipsoid currently: Airy1830. (Denoted by _1)
    lat_1 = lat - VII * dE**2 + VIII * dE**4 - IX * dE**6
    lon_1 = lon0 + X * dE - XI * dE**3 + XII * dE**5 - XIIA * dE**7

    # Want to convert to the GRS80 ellipsoid.
    # First convert to cartesian from spherical polar coordinates
    H = 0  # Third spherical coord.
    x_1 = (nu / F0 + H) * cos(lat_1) * cos(lon_1)
    y_1 = (nu / F0 + H) * cos(lat_1) * sin(lon_1)
    z_1 = ((1 - e2) * nu / F0 + H) * sin(lat_1)

    # Perform Helmut transform (to go between Airy 1830 (_1) and GRS80 (_2))
    s = -20.4894 * 10**-6  # The scale factor -1
    # The translations along x,y,z axes respectively
    tx, ty, tz = 446.448, -125.157, + 542.060
    # The rotations along x,y,z respectively, in seconds
    rxs, rys, rzs = 0.1502,  0.2470,  0.8421
    rx, ry, rz = rxs * pi / (180 * 3600.), rys * pi / \
        (180 * 3600.), rzs * pi / (180 * 3600.)  # In radians
    x_2 = tx + (1 + s) * x_1 + (-rz) * y_1 + (ry) * z_1
    y_2 = ty + (rz) * x_1 + (1 + s) * y_1 + (-rx) * z_1
    z_2 = tz + (-ry) * x_1 + (rx) * y_1 + (1 + s) * z_1

    # Back to spherical polar coordinates from cartesian
    # Need some of the characteristics of the new ellipsoid
    # The GSR80 semi-major and semi-minor axes used for WGS84(m)
    a_2, b_2 = 6378137.000, 6356752.3141
    # The eccentricity of the GRS80 ellipsoid
    e2_2 = 1 - (b_2 * b_2) / (a_2 * a_2)
    p = sqrt(x_2**2 + y_2**2)

    # Lat is obtained by an iterative proceedure:
    lat = arctan2(z_2, (p * (1 - e2_2)))  # Initial value
    latold = 2 * pi
    while abs(lat - latold) > 10**-16:
        lat, latold = latold, lat
        nu_2 = a_2 / sqrt(1 - e2_2 * sin(latold)**2)
        lat = arctan2(z_2 + e2_2 * nu_2 * sin(latold), p)

    # Lon and height are then pretty easy
    lon = arctan2(y_2, x_2)
    H = p / cos(lat) - nu_2

    # Uncomment this line if you want to print the results
    # print([(lat-lat_1)*180/pi, (lon - lon_1)*180/pi])

    # Convert to degrees
    lat = lat * 180 / pi
    lon = lon * 180 / pi

    # Job's a good'n.
    return lat, lon

In [ ]:
#@title Finding the long/lat coordinates for the places in the database
def setFindsGeo(brit_coin_finds, coin_finds):		# sets coordinates for all coin finds
	##### sorting coordinates #####
	geourl = "https://api.postcodes.io/places?q="

	# manual fixes for areas that aren't found by the geolocator service
	parishes = {"Savernake":"Cadley", 'Orford and Tunstall':'Orford', 'Exmoor':'Devon',
				'Ickworth':'Horringer', 'Bury St. Edmunds':'Blackthorpe'}
	districts = {'Bath and North East Somerset': "Bath", "City of Bristol":"Bristol", "Derbyshire Dales":"Longcliffe",
				"North Dorset":"Shillingstone", "Dorset":"Dorchester", "Weymouth and Portland":"Weymouth", 
				"Gravesham":"Cobham", "Medway":"Chattenden", "King's Lynn and West Norfolk":"King's Lynn",
				"Wiltshire":"Shrewton", 'City of Plymouth':'Plymouth', 'North Devon': 'Devon'}
	counties = {'Buckinghamshire': 'Aylesbury', "Norfolk":"Norwich", "Dorset":"Dorchester", 
				'Northamptonshire':'Northampton', 'Greater London Authority':'London', 
				'Cambridgeshire':'Cambridge'}

	# gets the coordinates for the places listed, at varying levels of precision
	# 28.8.17 - BUG: the service returns a list of places, and this prob. selects the first
	for i in range(len(coin_finds)):	# convers the UK geographic system to coordinates
		if (brit_coin_finds['easting'].iloc[i] == brit_coin_finds['easting'].iloc[i]):
			temp = OSGB36toWGS84(brit_coin_finds['easting'].iloc[i], brit_coin_finds['northing'].iloc[i])
			coin_finds.at[i, 'lat'] = temp[0]
			coin_finds.at[i, 'long'] = temp[1]
			
		else:	# get an estimate about the location based on the available data
			if (brit_coin_finds.iloc[i]['parish'] == brit_coin_finds.iloc[i]['parish']):
				if brit_coin_finds.iloc[i]['parish'] in parishes: add = parishes[brit_coin_finds.iloc[i]['parish']]
				else: add = brit_coin_finds.iloc[i]['parish']
			elif (brit_coin_finds.iloc[i]['district'] == brit_coin_finds.iloc[i]['district']):
				if brit_coin_finds.iloc[i]['district'] in districts: add = districts[brit_coin_finds.iloc[i]['district']]
				else: add = brit_coin_finds.iloc[i]['district']
			elif (brit_coin_finds.iloc[i]['county'] == brit_coin_finds.iloc[i]['county']):
				if brit_coin_finds.iloc[i]['county'] in counties: add = counties[brit_coin_finds.iloc[i]['county']]
				else: add = brit_coin_finds.iloc[i]['county']
			else:
				add = "Whalley"	# center of UK
			r = requests.get(geourl + add)
			temp = json.loads(r.text)
			try:
				coin_finds.at[i, 'lat'] = temp['result'][0]['latitude']
				coin_finds.at[i, 'long'] = temp['result'][0]['longitude']
			except:
				print(add)	# this should not print anything
				print(brit_coin_finds.iloc[i]['county'])
				print(brit_coin_finds.iloc[i]['district'])
				print(brit_coin_finds.iloc[i]['parish'])
		

	brit_coin_finds['certainty'] = 'highest'
	brit_coin_finds.loc[pd.isnull(brit_coin_finds.parish), 'certainty'] = 'lower'
	brit_coin_finds.loc[pd.isnull(brit_coin_finds.county), 'certainty'] = 'lowest'
	coin_finds['certainty'] = brit_coin_finds['certainty']

	return coin_finds

In [ ]:
# this function verifies that the number of reported identified coins found in the hoard
# is the same as the sum of the identified number of coin groups
def testing_database_connections(): 		# testing function, not normally used
	# this block creates a new DF, datatest, and initializes the sum of its coin groups to 0
	list_of_fields = ['old_findID', 'QuantityCoins', 'Denomination_KnownTotal', 'Denomination_UnknownTotal']
	datatest = brit_coin_finds[list_of_fields]
	datatest['sum_coin_groups'] = 0
	datatest.set_index('old_findID', inplace=True)
	
	# this sums up all the coin groups and puts the result in hoards, a dict
	hoards = {}
	for i in range(len(brit_coin_groups)):
		if brit_coin_groups.hoard.iloc[i] in hoards:
			hoards[brit_coin_groups.hoard.iloc[i]] += brit_coin_groups.quantity.iloc[i]
		else:
			hoards[brit_coin_groups.hoard.iloc[i]] = brit_coin_groups.quantity.iloc[i]
	
	# this block updates the datatest based on the dict we've just created
	hoard_list = datatest.index.tolist()
	temp_hoards = hoards.copy()		#This begins with 2424 hoards
	for i in temp_hoards:
		if i in hoard_list:
			try:
				datatest.at[i, 'sum_coin_groups'] = hoards[i]
			except:
				print('Hoard with no datable coin groups:', i, hoards[i])
			del hoards[i]
	
	# gives feedback to user
	#print(datatest[datatest.sum_coin_groups != 0])
	print('Overall {} hoards with {} coins.'.\
			format(len(hoard_list), datatest['QuantityCoins'].sum()))		
			# 431455 coins in 820 relevant coin groups
	print('{} hoards with {} coins are datable.'.\
			format(len(datatest[datatest.sum_coin_groups != 0]), datatest['sum_coin_groups'].sum()))
			# 323223 coins in 692 datable coin groups
	print(len(hoards))	# 1684 hoards remain here, meaning they are not relevant
	#print(datatest[datatest.Denomination_KnownTotal + datatest.Denomination_UnknownTotal != datatest.QuantityCoins])
	# 1 hoard has an issue here
	#print(len(datatest[datatest.sum_coin_groups != datatest.QuantityCoins]),
	#	datatest[datatest.sum_coin_groups != datatest.QuantityCoins])
	# 259 hoards have unknown denominations

In [ ]:
#@title Year limit - a minor function to help with cleaning Coin Groups
def year_limit(denom_list, denom, time):	# checks for relevant denominations
	if denom in denom_list:
		if time == "start": return denom_list[denom][0]
		if time == "end": return denom_list[denom][1]
	else: 
		print(denom)
		return -1

In [ ]:
#@title Update the coin groups to a specific date
def reg_update_coin_groups(year, month, day, coingroupsDB):
	entries_to_update = coingroupsDB.copy()
	update_from = dt.datetime(year, month, day)
	for i in coingroupsDB.index:
		if coingroupsDB.at[i, 'updated'] == coingroupsDB.at[i, 'updated']:
			tempTime = dt.datetime.strptime(coingroupsDB.at[i, 'updated'][:10], '%Y-%m-%d')
		else:
			tempTime = dt.datetime.strptime(coingroupsDB.at[i, 'created'][:10], '%Y-%m-%d')
		if tempTime < update_from:		# keep only the new/updated entries 
			entries_to_update = entries_to_update.drop(i)	# removes all the older entries	
	return entries_to_update

In [ ]:
#@title Setting Coin Finds
# takes the PAS scheme of coin finds and translates it to FLAME's
def setting_coin_finds(brit_coin_finds):
  cols_finds = ['hoard_id', 'hoard_gis_id', 'unique_id', 'name', 'startDate', 'endDate', 'type_find', 'hoard?', 'excavation?', 'single?', 'num_coins', 'num_known_coins', 'num_unknown_coins', 'year_found',
		'year_end_found', 'comments', 'lat', 'long', 'certainty', 'owner', 'created', 'imported']
  coin_finds = pd.DataFrame(columns=cols_finds)
  coin_finds['hoard_id'] = brit_coin_finds['ID no.']
  coin_finds['hoard_gis_id'] = brit_coin_finds['GIS_ID']
  coin_finds['unique_id'] = brit_coin_finds['old_findID']
	# name is made of PAS: <location name> + <ID number in PAS>
  try:
    coin_finds['name'] = 'PAS: ' + brit_coin_finds['knownas'].map(str) + ' ' + brit_coin_finds['ID no.'].map(str)
  except:
    coin_finds['name'] = 'PAS: ' + brit_coin_finds['knownas'].map(str) + ' ' + brit_coin_finds['GIS_ID'].map(str) 
  coin_finds['type_find'] = brit_coin_finds['DatasetQual']
  coin_finds['startDate'] = brit_coin_finds['fromTerminalYear']
  coin_finds['endDate'] = brit_coin_finds['toTerminalYear']
  coin_finds['num_coins'] = brit_coin_finds['QuantityCoins']
  coin_finds['num_known_coins'] = brit_coin_finds['Denomination_KnownTotal']
  coin_finds['num_unknown_coins'] = brit_coin_finds['Denomination_UnknownTotal']
  coin_finds['year_found'] = brit_coin_finds['YearFound1']
  coin_finds['year_end_found'] = brit_coin_finds['YearFound2']
  coin_finds['comments'] = brit_coin_finds['description']
  coin_finds['imported'] = dt.datetime.now()
  coin_finds['owner'] = 'PAS UK Finds'
  coin_finds['hoard?'] = 'hoard'
  coin_finds.loc[coin_finds.type_find == 'AC_Excavated', 'excavation?'] = 'excav'
  return coin_finds

In [ ]:
#@title Setting Coin Groups
# this function translates the PAS Coin Groups to FLAME's format
def setting_coin_groups(brit_coin_groups):
	cols = ['hoard_id', 'coin_group_id', 'start_year', 'end_year', 'revised_start', 'revised_end', 'ruler', 'revised_ruler',
		'denomination', 'num_coins', 'mint', 'imported', 'created', 'updated']
	coin_groups = pd.DataFrame(columns=cols)

	coin_groups['hoard_id'] = brit_coin_groups['hoardID']
	coin_groups['coin_group_id'] = brit_coin_groups['id']
	coin_groups['start_year'] = brit_coin_groups['fromDate']
	coin_groups['end_year'] = brit_coin_groups['toDate']
	coin_groups['denomination'] = brit_coin_groups['denomination']
	coin_groups['num_coins'] = brit_coin_groups['quantity']
	coin_groups['mint'] = brit_coin_groups['mint']
	coin_groups['created'] = brit_coin_groups['created']
	coin_groups['ruler'] = brit_coin_groups['ruler']
	coin_groups['updated'] = brit_coin_groups['updated']
	coin_groups['imported'] = dt.datetime.now()
	return coin_groups

In [ ]:
#@title Initial filtering of denominations, rulers, mints and specific entries
def initial_filtering(coin_groups_raw):		# filters out irrelevant denominations, rulers, mints, and entries
	have_bad_dates = coin_groups_raw[coin_groups_raw.end_year < 325]	# works

	list_of_bad = have_bad_dates.index.tolist()	# prepare for filtering out
	coin_groups_filtered = coin_groups_raw.drop(list_of_bad)	# drop all irrelevant rows

	irrelevant_denominations = ['Radiate (antoninianus)', 'Sestertius', 'Denarius (Empire)', 
								'Denarius (Roman Republic)', 'Dupondius', 'Quinarius',
								'As (Roman Republic)', 'Quadrans (Roman Republic)', 'Quadrans',
								'Sestertius (Roman Republic)', 'Dupondius or as', 'Double sestertius',
								'Sestertius, dupondius or as', 'Q radiate', 'As', 'Semis', 'Stater (gold)',
								'Aureus (Republic/Empire)', 'Denarius', 'Stater (silver)', 
								'Quarter stater (gold)', 'Quarter stater (silver)', 'Half stater', 'Denarius',
								'As', 'Denarius (Empire)', 'Unit (copper alloy)', 'Drachm', 'Tetradrachm',
								'Potin (Cast bronze)']
	for irr_den in irrelevant_denominations:
		print('{} of "{}" removed.'.format(len(coin_groups_filtered[coin_groups_filtered.denomination == irr_den]), irr_den))
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.denomination != irr_den]

	print('Overall, {} coin groups remain in the database and ready for import'.format(len(coin_groups_filtered)))
	print('This is the list of coin denominations that remains in the database: {}'.format(coin_groups_filtered.denomination.unique()))
	print()

	# Removes all irrelevant rulers from the data frame
	irrelevant_rulers = ["Julio-Claudian (uncertain)", "Caligula", "Claudius", "Vespasian", "Marcus Aurelius (as Augustus)",
		"Lucilla", "Antonine Empress, uncertain, 138-185", "Uncertain - 1st/2nd Century AD", 'Licinius I', 
		'Diocletian', 'Constantius I', 'Maximian I', 'Galeria Valeria', 'Nero', 'Galba', 'Otho', 'Vitellius',
		'Titus', 'Trajan', 'Hadrian', 'Sabina', 'Nerva', 'Domitian', 'Antoninus Pius', 'Aelius Caesar', 'Maxentius',
		'Marciana', 'Maximinus Daia', 'Tetrarchic Ruler (uncertain issuer)', 'Faustina II', 'Caracalla', 'Philip I',
		'Philip II', 'Gordian III', 'Severus Alexander', 'Elagabalus', 'Uncertain - 1st-mid 3rd century', 'Probus',
		'Victorinus', 'Galerius', 'Divus Claudius (Official)', 'Aurelian', 'Gallienus (sole reign)', 
		'Salonina (sole reign of Gallienus)', 'Radiate, Uncertain Ruler 260-296', 'Balbinus', 'Licinius II', 
		'Radiate (Barbarous)', 'Balbinus', 'Julia Maesa', 'Severus', 'Carausius', 'Augustus', 'Hostilian',
		'Tiberius', 'Postumus', 'Maximinus', 'Juba I', 'Tranquillina', 'Trebonius Gallus', 'Trajan Decius',
		'Trajan', 'Julia Domna', 'ECEN/ECE', 'AESV', 'Uninscribed', 'Cunobelin', 'Gelon I And Hieron I',
		'Valerian I', 'Epaticcus', 'Gallienus', 'Commodus', 'Claudius II', 'Carinus', 
		'VOLISIOS DUMNOVELLAUNOS', 'VOLISIOS CARTIVELLAVNOS', 'SAENV', 'IISVPRASV', 'DUMNOCO TIGIR SENO',
		'Gallienus', 'Otacilia', 'Tincomarus', 'Verica', 'Julia Mamaea', 'CARA[TACUS]',
		'EISV', 'AVN COST', 'ANTED (Iceni)', 'CANI DVRO', 'ANTED (Dobunni)', 'Eppillus', 
		'VOLISIOS DUMNOCOVEROS', 'Salonina', 'VEP CORF', 'DVMNOCO TIGIR SENO', 'BODVOC', 
		'SVB ESVPRASTO ESICO FECIT']
	for irr_ruler in irrelevant_rulers:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.ruler != irr_ruler]

	irrelevant_mints = ['Ostia']
	for irr_mint in irrelevant_mints:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.mint != irr_mint]

	# manual removal of entries
	irrelevant_entries = [21109, 1390, 1391, 6462, 6929, 6953, 6954, 6977, 6983, 6989, 8825, 
							8827, 9064, 9329, 9503, 10488, 11438] 
							# nothing/very little known about these coins and error in ruler attribution
	for irr_entry in irrelevant_entries:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.coin_group_id != irr_entry]

	return coin_groups_filtered

In [ ]:
def coin_group_cleaning(coin_groups, flame_rulers):
	#flame_denominations = pd.read_excel('Denominations.xlsx')
	#flame_mints = pd.read_excel('Mints.xlsx')
	#flame_rulers = pd.read_excel('Rulers.xlsx')

	# setting conversions
	ruler_list = {"House of Constantine":(307, 363), "House of Valentinian":(364,378), "House of Theodosius":(378, 408),
		"Magnentius":(350,353), "Uncertain (AD 260 - 402)":(260, 402), 
		'Uncertain - 4th century':(300, 399), 'Magnentius or Decentius': (350, 353),
		'Flavius Victor':(384, 388), 'Dalmatius':(335, 337),
		'Constantinopolis':(330, 341),	# based on the existing entries in the database
		'Procopius': (365, 366),
		'Constans': (330, 350),
		'Fausta': (324, 330),			# based on the existing entries in the database
		'Urbs Roma': (330, 341),		# based on the existing entries in the database
		'Crispus': (311, 330),			# based on the existing entries in the database
		'Theodora': (337, 341),			# based on the existing entries in the database
		'Magnus Maximus or Flavius Victor': (384, 388),
		'Helena': (324, 341),
		'Constantius Gallus': (351, 355), 
		'Decentius': (350, 353),
		'Honorius (emperor)': (395, 423),
		'Nummus, uncertain ruler, c. 330-402': (330, 402)
		}
	for i in range(len(flame_rulers)):
		ruler_list[flame_rulers.at[i, 'RulerName']] = (flame_rulers['RulerStartYear'].iloc[i],flame_rulers['RulerEndYear'].iloc[i])

	denomination_dates = {"Nummus (AE 1 - AE 4)": (302, 402),	# based on existing entries
						"Radiate or nummus": (260, 402),		# based on existing entries (/w corrections)
						"Siliqua": (360, 402),					# based on existing entries
						"Uncertain (copper alloy)":(-100, 410),	# one such entry
						"Uncertain (silver)": (-100, 410),		# one such entry
						"Unspecified ruler (contemporary copy)":(-100, 410), # one such entry
						"Nummus, uncertain ruler, c. 330-402": (300, 402)
						}
	# order: 	mint name in UK database:mint name in FLAME
	mint_conversion = {"Trier": "Colonia Augusta Treverorum", "Lyon": "Lugdunensium", "Lugdunum":"Lugdunensium",  
					"Rome": "Roma",	"Thessalonica": "Thessalonika", "Siscia": "Siscia", "Aquileia": "Aquileia", 
					"Milan": "Mediolanum", 'Amiens (Ambianum)': 'Ambianum', 'Nicomedia':'Nikomedia', 
					"Heraclea":"Heraclea", "London":"Londinium", "Antioch":"Antioch", "Arles": "Arelato",
					"Pavia":"Ticinum", "Cyzicus":"Kyzikos", 'Sirmium':'Sirmium', 
					'Constantinople':'Constantinople', "Ravenna": "Ravenna", "Alexandria": "Alexandria",
					'Unattributed': 'Unknown', 'Eastern mint': 'Unknown (East Roman)', 
					'Gallic mint': 'Unknown (Gaul)', 'Arles or Lyons ': 'Arelato or Lugdunum',
					'Italian mint': 'Unknown (Italy)', 'Alexandria (Egypt)': 'Alexandria'}
	# order: 	denomination in UK database:denomination name in FLAME
	denomination_conversion = {'Nummus (AE 1 - AE 4)':"AE 1-4 (UK find)",	# new denomination (bronze)
							'Miliarensis':"miliarensis",
							'Siliqua': "siliqua",
							'Radiate or nummus':"radiate or nummus (UK find)", # new denomination (bronze)
							'Solidus':"solidus",
							'Half unit (silver)':"half unit", 	# new denomination (silver)
							'Unit (silver)': "unit",			# new denomination (silver)
							'Half-siliqua':"1/2 siliqua",
							'Tremissis':"tremissis",
							'Uncertain':"uncertain",
							'Uncertain (gold)':"uncertain (gold)",		# new denomination (gold)
							'Uncertain (silver)':"uncertain (silver)",	# new denomination (silver)
							'Uncertain (copper alloy)':"unidentified bronze coins"
							}
	ruler_conversion = {'Honorius (emperor)': 'Honorius',			# add at least some of this to FLAME database
						'Nummus, uncertain ruler, c. 330-402': 'Unknown',
						'Uncertain': 'Unknown',
						'Uncertain - 4th century': 'Unknown',
						'Uninscribed': 'Unknown',
						'Unattributed': 'Unknown',
						'Dalmatius': 'Constantine I',
						'Magnentius or Decentius': 'Magnentius or Decentius',
						'Flavius Victor': 'Flavius Victor', 
						'Dalmatius': 'Dalmatius',
						'Constantinopolis': 'Constantinopolis',	
						'Procopius': 'Procopius',
						'Constans': 'Constans I',
						'Fausta': 'Fausta',			
						'Urbs Roma': 'Roma',		
						'Crispus': 'Crispus',			
						'Theodora': 'Theodora (4th century)',
						'Magnus Maximus or Flavius Victor': 'Magnus Maximus or Flavius Victor',
						'Helena': 'Helena',
						'Constantius Gallus': 'Constantius Gallus', 
						'Decentius': 'Decentius'
						}

	coin_groups['revised_start'] = coin_groups['start_year']
	coin_groups['revised_end'] = coin_groups['end_year']
	coin_groups['revised_ruler'] = coin_groups['ruler']
	rulers_to_resolve = set()
	
	for i in coin_groups.index:
		# this updates revised_ruler based on the ruler_conversion dictionary. This is for display (not data extraction)
		ruler_to_check = coin_groups.at[i,'ruler']
		if ruler_to_check in ruler_conversion:
			coin_groups.at[i, 'revised_ruler'] = ruler_conversion[coin_groups.at[i,'ruler']]

		# this section fills in the dates based on ruler (or denomination) if they are missing
		if (coin_groups.at[i, 'revised_start'] != coin_groups.at[i, 'revised_start']):
			temp_start = -1
			if coin_groups.at[i, 'ruler'] != 'Unspecified ruler (contemporary copy)':
				try:
					temp_start = ruler_list[coin_groups.at[i, 'ruler']][0]
				except:
					if coin_groups.at[i, 'ruler'] not in ruler_conversion:
						#print("Error: Unknown ruler: {}".format(coin_groups.at[i, 'ruler']))
						rulers_to_resolve.add(coin_groups.at[i, 'ruler'])
			elif coin_groups.at[i, 'ruler'] == 'Unspecified ruler (contemporary copy)':
				temp_start = year_limit(denomination_dates, coin_groups.at[i, 'denomination'], "start")
			if temp_start != -1: coin_groups.at[i, 'revised_start'] = temp_start

		if (coin_groups.at[i, 'revised_end'] != coin_groups.at[i, 'revised_end']):
			temp_end = -1
			if coin_groups.at[i, 'ruler'] != 'Unspecified ruler (contemporary copy)':
				try:
					temp_end = ruler_list[coin_groups.at[i, 'ruler']][1]
				except:
					if coin_groups.at[i, 'ruler'] not in ruler_conversion:
						#print("Error: Unknown ruler: {}".format(coin_groups.at[i, 'ruler']))
						rulers_to_resolve.add(coin_groups.at[i, 'ruler'])
			elif coin_groups.at[i, 'ruler'] == 'Unspecified ruler (contemporary copy)':
				temp_end = year_limit(denomination_dates, coin_groups.at[i, 'denomination'], "end")
			if temp_end != -1: coin_groups.at[i, 'revised_end'] = temp_end


		# this section standardizes the mint names to those in FLAME's database
		try:
			if coin_groups.at[i, 'mint'] == coin_groups.at[i, 'mint']:
				try:
					coin_groups.at[i, 'mint'] = mint_conversion[coin_groups.at[i, 'mint']]
				except:
					print("Error: Unknown mint: {}".format(coin_groups.at[i, 'mint']))	# this should print nothing if working as intended
		except:
			continue

		# this section standardizes the denomination names to those in the FLAME database
		try:
			if coin_groups.at[i, 'denomination'] == coin_groups.at[i, 'denomination']:
				try:
					coin_groups.at[i, 'denomination'] = denomination_conversion[coin_groups.at[i, 'denomination']]
				except:
					print("Error: Unknown denomination: {}".format(coin_groups.at[i, 'denomination']))
		except:
			continue

	if len(rulers_to_resolve) > 1:	# the only entry here should be nan, otherwise print it all
		print("The unknown rulers are: {}".format(rulers_to_resolve))

	return coin_groups

<h2>Import the raw data files (based on the ones received in 2016)<h2>

In [ ]:
_ = files.upload()
brit_imported_coin_finds = pd.read_excel('Consolidated Reece 16+ hoard details_with numbers.xlsx')
brit_imported_coin_finds.head(1)

Saving Consolidated Reece 16+ hoard details_with numbers.xlsx to Consolidated Reece 16+ hoard details_with numbers.xlsx


,ID no.,GIS_ID,DatasetQual,old_findID,lastRuler,reeceID,fromTerminalYear,toTerminalYear,description,EG_Notes,AC_Notes,QuantityCoins,QuanityCoins_Qualifier,QuanityCoins_CopperAlloy,QuanityCoins_BaseSilver,QuanityCoins_Silver,QuanityCoins_Gold,QuanityCoins_Uncertain,QuanityCoins_KnownTotal,QuanityCoins_UnknownTotal,Denomination_As,Denomination_As_RR,Denomination_Aureus_RE,Denomination_Denarius,Denomination_Denarius_Empire,Denomination_Denarius_RR,Denomination_DoubleSestertius,Denomination_Drachm,Denomination_Dupondius,Denomination_DupondiusAs,Denomination_Halfstater,Denomination_Halfunit_silver,Denomination_Halfsiliqua,Denomination_Miliarensis,Denomination_Minim_Silver,Denomination_Nummus_AE1_AE4,Denomination_Potin_CastBronze,Denomination_Qradiate,Denomination_Quadrans,Denomination_Quadrans_RR,...,Denomination_Radiate_Antoninianus,Denomination_Radiate_nummus,Denomination_Semis,Denomination_Sestertius,Denomination_Sestertius_RR,Denomination_SestertiusDupondiusAs,Denomination_Siliqua,Denomination_Solidus,Denomination_Stater_CopperAlloy,Denomination_Stater_Gold,Denomination_Stater_Silver,Denomination_Tetradrachm,Denomination_Tremissis,Denomination_Uncertain,Denomination_Uncertain_CA,Denomination_Uncertain_Gold,Denomination_Uncertain_Silver,Denomination_Unit_CopperAlloy,Denomination_Unit_Silver,Denomination_UnspecifiedRuler_CCopy,Denomination_KnownTotal,Denomination_UnknownTotal,quantityArtefacts,quantityContainers,YearFound1,YearFound2,Country,county,district,parish,knownas,PAS_Region,CS_Region,CS_Hoard,NGR,easting,northing,Latitude,Longitude,Altitude
0,4,2083,EG_IARCH,IARCH-15228D,Magnus Maximus,20,383.0,388.0,"Robertson 2000, 358 no. 1474:""The hoard of 255...","Images in file. BM registration 1984,0221.1 to...",NaN,255.0,NaN,NaN,NaN,255.0,NaN,NaN,255,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,0,NaN,NaN,1983.0,1983.0,England,Bath and North East Somerset,Bath and North East Somerset,Newton St. Loe,Newton Mills,SouthWest,SouthWestern,NaN,ST7165,371000.0,165000.0,NaN,NaN,NaN


In [ ]:
_ = files.upload()
brit_imported_coin_groups = pd.read_csv('Roman hoards content summaries_short.csv')
brit_imported_coin_groups.head(1)

Saving Roman hoards content summaries_short.csv to Roman hoards content summaries_short.csv


,id,hoard,hoardID,broadperiod,ruler,denomination,fromDate,toDate,mint,geography,quantity,created,createdBy,updated,updatedBy
0,26878,BM-0D5E3C,3252,ROMAN,Tetrarchic Ruler (uncertain issuer),Nummus (AE 1 - AE 4),298.0,299.0,Trier,NaN,1.0,2015-10-16T11:49:38Z,26767,2015-10-16T11:55:42Z,26767.0


In [ ]:
_ = files.upload()
flame_ruler_list = pd.read_excel('Rulers.xlsx')

Saving Rulers.xlsx to Rulers.xlsx


<h3>Begin cleaning Finds and Groups of the original dataset</h3>



In [ ]:
finds = setting_coin_finds(brit_imported_coin_finds)
finds = setFindsGeo(brit_imported_coin_finds, finds)
#finds.head(5)
groups = setting_coin_groups(brit_imported_coin_groups)
#groups.head(5)

In [ ]:
groups = initial_filtering(groups)
groups = coin_group_cleaning(groups, flame_ruler_list)

4890 of "Radiate (antoninianus)" removed.
429 of "Sestertius" removed.
2356 of "Denarius (Empire)" removed.
59 of "Denarius (Roman Republic)" removed.
114 of "Dupondius" removed.
2 of "Quinarius" removed.
1 of "As (Roman Republic)" removed.
1 of "Quadrans (Roman Republic)" removed.
2 of "Quadrans" removed.
1 of "Sestertius (Roman Republic)" removed.
157 of "Dupondius or as" removed.
1 of "Double sestertius" removed.
232 of "Sestertius, dupondius or as" removed.
10 of "Q radiate" removed.
182 of "As" removed.
2 of "Semis" removed.
0 of "Stater (gold)" removed.
42 of "Aureus (Republic/Empire)" removed.
0 of "Denarius" removed.
0 of "Stater (silver)" removed.
0 of "Quarter stater (gold)" removed.
0 of "Quarter stater (silver)" removed.
0 of "Half stater" removed.
0 of "Denarius" removed.
0 of "As" removed.
0 of "Denarius (Empire)" removed.
0 of "Unit (copper alloy)" removed.
0 of "Drachm" removed.
0 of "Tetradrachm" removed.
0 of "Potin (Cast bronze)" removed.
Overall, 7689 coin groups re

In [ ]:
groupB = reg_update_coin_groups(2015, 6, 1, groups)		# change these numbers (year\month\date) to update all subsequent entries
groups.head()

,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated
3,3020,19070,337.0,NaN,337.0,363.0,House of Constantine,House of Constantine,AE 1-4 (UK find),7.0,NaN,2021-05-01 13:14:01.540632,2015-02-11T13:14:54Z,NaN
4,3020,19071,NaN,NaN,302.0,402.0,Unspecified ruler (contemporary copy),Unspecified ruler (contemporary copy),AE 1-4 (UK find),3.0,NaN,2021-05-01 13:14:01.540632,2015-02-11T13:15:12Z,2015-02-11T13:16:04Z
5,3119,26982,364.0,367.0,364.0,367.0,House of Valentinian,House of Valentinian,AE 1-4 (UK find),24.0,NaN,2021-05-01 13:14:01.540632,2016-02-02T11:27:14Z,NaN
6,3119,26983,367.0,375.0,367.0,375.0,House of Valentinian,House of Valentinian,AE 1-4 (UK find),87.0,NaN,2021-05-01 13:14:01.540632,2016-02-02T11:27:32Z,NaN
7,3119,26984,375.0,378.0,375.0,378.0,House of Valentinian,House of Valentinian,AE 1-4 (UK find),4.0,NaN,2021-05-01 13:14:01.540632,2016-02-02T11:27:51Z,NaN


<h2>Import additional files, to be merged with the data above</h2>

In [ ]:
_ = files.upload()
ids = pd.read_csv('df_pas_hoards.csv')

In [ ]:
_ = files.upload()
hoards = pd.ExcelFile('HoardsTableForRory.xlsx')

hoards_table = hoards.parse(9)
for i in range(9):
    hoards_table = pd.concat([hoards_table, hoards.parse(i)])

hoards_table.head(1)

Saving HoardsTableForRory.xlsx to HoardsTableForRory.xlsx


,GIS_ID,DatasetQual,old_findID,HoardQual,broadperiod,lastRuler,DeJerseyPhase_Start,DeJerseyPhase_End,DeJerseyPhase_Qual,BroadIAPhase,reeceID,fromTerminalYear,toTerminalYear,terminalReason,description,EG_Notes,AC_Notes,qualityRatingNumismatic,TID,QuantityCoins,QuanityCoins_Qualifier,QuanityCoins_CopperAlloy,QuanityCoins_BaseSilver,QuanityCoins_Silver,QuanityCoins_Gold,QuanityCoins_Uncertain,QuanityCoins_KnownTotal,QuanityCoins_UnknownTotal,Denomination_As,Denomination_As_RR,Denomination_Aureus_RE,Denomination_Denarius,Denomination_Denarius_Empire,Denomination_Denarius_RR,Denomination_DoubleSestertius,Denomination_Drachm,Denomination_Dupondius,Denomination_DupondiusAs,Denomination_Halfstater,Denomination_Halfunit_silver,...,HoardCat_Lead Alloy,HoardCat_WhiteMetal,HoardCat_Jet_Shale,HoardCat_Gem,HoardCat_Base Silver,HoardCat_Tin_TinAlloy,HoardCat_Enamel,HoardCat_Mortar_Plaster,HoardCat_Other,HoardCat_Copper,HoardCat_Total,AssMatCul_Qual,AssMatCul_Bone_Animal,AssMatCul_Bone_Human,AssMatCul_Bone_Objects,AssMatCul_Brooches,AssMatCul_CA_Vessel,AssMatCul_CA_Other,AssMatCul_Bronze_Objects,AssMatCul_Ceramic_Vessels,AssMatCul_Ceramic_Other,AssMatCul_PW_VesselsPlates,AssMatCul_SV_VesselsPlates,AssMatCul_SV_Other,AssMatCul_Gold,AssMatCul_Lead,AssMatCul_Iron,AssMatCul_uncertainMetal,AssMatCul_Wood_Vessels,AssMatCul_Wood_Other,AssMatCul_Glass,AssMatCul_ShaleJet,AssMatCul_Leather,AssMatCul_Stone,AssMatCul_Plantfibre,AssMatCul_Ivory,AssMatCul_Other,AssMatCul_SumCat,AssMatCul_VesselCount,AssMatCult_Notes
0,57,AC_Excavated,IARCH-99712F,NaN,ROMAN,"Antonine Empress, uncertain, 138-185",NaN,NaN,NaN,NaN,NaN,138.0,180.0,NaN,"West Parade site Area III, Roman interval towe...",Hoard identified from report by AC - coin deta...,N.B. NGR based on site plan & Magic Map. Stee...,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,3,0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Yes,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


In [ ]:
#@title Fill hoards_table's IDs for web
# this section introduces an id (for web) to the different hoards in the table
hoards_table = hoards_table.reset_index(drop=True)
old_find_list = list(hoards_table['old_findID'])
unique_id = 'IARCH-99712F'
list_of_ids = []
for i in range(len(hoards_table)):
  unique_id = hoards_table.at[i, 'old_findID']
  try:
    id_no = int(ids[ids['unique_id'] == unique_id]['hoard_id'])
    list_of_ids.append(id_no)
  except:
      list_of_ids.append(-1)
      #print('Error with this unique_id: ' + str(unique_id))

hoards_table['ID no.'] = list_of_ids
hoards_table.head(3) # this df should be merged with the 'brit_imported_coin_finds' df

#testing: 
#i = 0
#for duo in zip(old_find_list, list_of_ids):
#  print(duo)
#  i += 1
#  if i > 5: break

,GIS_ID,DatasetQual,old_findID,HoardQual,broadperiod,lastRuler,DeJerseyPhase_Start,DeJerseyPhase_End,DeJerseyPhase_Qual,BroadIAPhase,reeceID,fromTerminalYear,toTerminalYear,terminalReason,description,EG_Notes,AC_Notes,qualityRatingNumismatic,TID,QuantityCoins,QuanityCoins_Qualifier,QuanityCoins_CopperAlloy,QuanityCoins_BaseSilver,QuanityCoins_Silver,QuanityCoins_Gold,QuanityCoins_Uncertain,QuanityCoins_KnownTotal,QuanityCoins_UnknownTotal,Denomination_As,Denomination_As_RR,Denomination_Aureus_RE,Denomination_Denarius,Denomination_Denarius_Empire,Denomination_Denarius_RR,Denomination_DoubleSestertius,Denomination_Drachm,Denomination_Dupondius,Denomination_DupondiusAs,Denomination_Halfstater,Denomination_Halfunit_silver,...,HoardCat_WhiteMetal,HoardCat_Jet_Shale,HoardCat_Gem,HoardCat_Base Silver,HoardCat_Tin_TinAlloy,HoardCat_Enamel,HoardCat_Mortar_Plaster,HoardCat_Other,HoardCat_Copper,HoardCat_Total,AssMatCul_Qual,AssMatCul_Bone_Animal,AssMatCul_Bone_Human,AssMatCul_Bone_Objects,AssMatCul_Brooches,AssMatCul_CA_Vessel,AssMatCul_CA_Other,AssMatCul_Bronze_Objects,AssMatCul_Ceramic_Vessels,AssMatCul_Ceramic_Other,AssMatCul_PW_VesselsPlates,AssMatCul_SV_VesselsPlates,AssMatCul_SV_Other,AssMatCul_Gold,AssMatCul_Lead,AssMatCul_Iron,AssMatCul_uncertainMetal,AssMatCul_Wood_Vessels,AssMatCul_Wood_Other,AssMatCul_Glass,AssMatCul_ShaleJet,AssMatCul_Leather,AssMatCul_Stone,AssMatCul_Plantfibre,AssMatCul_Ivory,AssMatCul_Other,AssMatCul_SumCat,AssMatCul_VesselCount,AssMatCult_Notes,ID no.
0,57,AC_Excavated,IARCH-99712F,NaN,ROMAN,"Antonine Empress, uncertain, 138-185",NaN,NaN,NaN,NaN,NaN,138.0,180.0,NaN,"West Parade site Area III, Roman interval towe...",Hoard identified from report by AC - coin deta...,N.B. NGR based on site plan & Magic Map. Stee...,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,3,0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Yes,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3242
1,76,AC_Excavated,IARCH-145D5D,NaN,ROMAN,Arcadius,NaN,NaN,NaN,NaN,NaN,383.0,408.0,Date of latest ruler/issuer,…hoard of over 115 3rd- & 4th-century coins (G...,No details appear to be published. There is a ...,"N.B. Quite steep SE facing slope, part of a NE...",1.0,NaN,115.0,NaN,115.0,NaN,NaN,NaN,NaN,115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,632
2,77,AC_Excavated,IARCH-19777D,NaN,ROMAN,Arcadius,NaN,NaN,NaN,NaN,NaN,383.0,402.0,Date of latest ruler/issuer,Ten nummi found during excav. of a round barro...,"Coins listed on Pastscape as of Constantine, V...",N.B. On SW facing slope of a promontory on Sal...,1.0,NaN,10.0,NaN,10.0,NaN,NaN,NaN,NaN,10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3121


In [ ]:
brit_imported_coin_finds.head(5)

,ID no.,GIS_ID,DatasetQual,old_findID,lastRuler,reeceID,fromTerminalYear,toTerminalYear,description,EG_Notes,AC_Notes,QuantityCoins,QuanityCoins_Qualifier,QuanityCoins_CopperAlloy,QuanityCoins_BaseSilver,QuanityCoins_Silver,QuanityCoins_Gold,QuanityCoins_Uncertain,QuanityCoins_KnownTotal,QuanityCoins_UnknownTotal,Denomination_As,Denomination_As_RR,Denomination_Aureus_RE,Denomination_Denarius,Denomination_Denarius_Empire,Denomination_Denarius_RR,Denomination_DoubleSestertius,Denomination_Drachm,Denomination_Dupondius,Denomination_DupondiusAs,Denomination_Halfstater,Denomination_Halfunit_silver,Denomination_Halfsiliqua,Denomination_Miliarensis,Denomination_Minim_Silver,Denomination_Nummus_AE1_AE4,Denomination_Potin_CastBronze,Denomination_Qradiate,Denomination_Quadrans,Denomination_Quadrans_RR,...,Denomination_Radiate_nummus,Denomination_Semis,Denomination_Sestertius,Denomination_Sestertius_RR,Denomination_SestertiusDupondiusAs,Denomination_Siliqua,Denomination_Solidus,Denomination_Stater_CopperAlloy,Denomination_Stater_Gold,Denomination_Stater_Silver,Denomination_Tetradrachm,Denomination_Tremissis,Denomination_Uncertain,Denomination_Uncertain_CA,Denomination_Uncertain_Gold,Denomination_Uncertain_Silver,Denomination_Unit_CopperAlloy,Denomination_Unit_Silver,Denomination_UnspecifiedRuler_CCopy,Denomination_KnownTotal,Denomination_UnknownTotal,quantityArtefacts,quantityContainers,YearFound1,YearFound2,Country,county,district,parish,knownas,PAS_Region,CS_Region,CS_Hoard,NGR,easting,northing,Latitude,Longitude,Altitude,certainty
0,4,2083,EG_IARCH,IARCH-15228D,Magnus Maximus,20,383.0,388.0,"Robertson 2000, 358 no. 1474:""The hoard of 255...","Images in file. BM registration 1984,0221.1 to...",NaN,255.0,NaN,NaN,NaN,255.0,NaN,NaN,255,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,0,NaN,NaN,1983.0,1983.0,England,Bath and North East Somerset,Bath and North East Somerset,Newton St. Loe,Newton Mills,SouthWest,SouthWestern,NaN,ST7165,371000.0,165000.0,NaN,NaN,NaN,highest
1,10,193,AC_Excavated,IARCH-CB0ABA,Honorius (emperor),21,393.0,402.0,"""At Camerton three Roman villas were discovere...",NaN,N.B. These are prob. not ‘villas’ but part of ...,26.0,NaN,NaN,NaN,26.0,NaN,NaN,26,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,NaN,1.0,1814.0,1814.0,England,Bath and North East Somerset,Bath and North East Somerset,Camerton,Camerton,SouthWest,SouthWestern,NaN,ST 688 562,368800.0,156200.0,NaN,NaN,NaN,highest
2,12,1553,EG_IARCH,IARCH-3BA394,Honorius (emperor),21,395.0,402.0,2009 T233 (25 siliquae); 2010 T746 (2 siliquae...,Original find: This small clipped siliqua hoar...,NaN,32.0,NaN,NaN,NaN,32.0,NaN,NaN,32,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,0,NaN,NaN,2009.0,2012.0,England,Wiltshire,Wiltshire,Pewsey,Pewsey,SouthWest,NaN,NaN,SU184562,418400.0,156200.0,NaN,NaN,NaN,highest
3,14,248,AC_Excavated,IARCH-40793E,House of Constantine,17,337.0,341.0,E.M. Besly listed 28 AE coins (Catsgore hoard ...,Report to be checked.,"N.B. Gentle lower SW facing slope of hill, clo...",28.0,NaN,28.0,NaN,NaN,NaN,NaN,28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,0,NaN,NaN,1971.0,1972.0,England,Somerset,South Somerset,Somerton,Catsgore,SouthWest,SouthWestern,NaN,ST 5083 2647,350830.0,126470.0,NaN,NaN,NaN,highest
4,15,1999,EG_IARCH,IARCH-4096EB,House of Valentinian,19,367.0,378.0,"Robertson 2000, 348 no. 1424:""""The spot [i.e. ...",NB the three miliarenses mentioned in VCH do n...,NaN,1496.0,NaN,NaN,NaN,1496.0,NaN,NaN,1496,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,

In [ ]:
# harmonize both Find dfs
brit_imported_coin_finds.head()
print(len(brit_imported_coin_finds)) # 820
all_hoard_numbers = set(hoards_table['old_findID'].values)
all_hoard_numbers2 = set(hoards_table['old_findID'].values)

for i in range(len(brit_imported_coin_finds)):
    if brit_imported_coin_finds['old_findID'].iloc[i] in all_hoard_numbers:
        all_hoard_numbers.discard(brit_imported_coin_finds['old_findID'].iloc[i])
print(len(all_hoard_numbers)) # 632, these are hoards not examined previously.

additional_hoards = hoards_table[hoards_table['old_findID'].isin(all_hoard_numbers)]
additional_hoards.head(2) # note that this has 180 columns compared to 82 in the brit_imported_coin_finds file

820
632


,GIS_ID,DatasetQual,old_findID,HoardQual,broadperiod,lastRuler,DeJerseyPhase_Start,DeJerseyPhase_End,DeJerseyPhase_Qual,BroadIAPhase,reeceID,fromTerminalYear,toTerminalYear,terminalReason,description,EG_Notes,AC_Notes,qualityRatingNumismatic,TID,QuantityCoins,QuanityCoins_Qualifier,QuanityCoins_CopperAlloy,QuanityCoins_BaseSilver,QuanityCoins_Silver,QuanityCoins_Gold,QuanityCoins_Uncertain,QuanityCoins_KnownTotal,QuanityCoins_UnknownTotal,Denomination_As,Denomination_As_RR,Denomination_Aureus_RE,Denomination_Denarius,Denomination_Denarius_Empire,Denomination_Denarius_RR,Denomination_DoubleSestertius,Denomination_Drachm,Denomination_Dupondius,Denomination_DupondiusAs,Denomination_Halfstater,Denomination_Halfunit_silver,...,HoardCat_WhiteMetal,HoardCat_Jet_Shale,HoardCat_Gem,HoardCat_Base Silver,HoardCat_Tin_TinAlloy,HoardCat_Enamel,HoardCat_Mortar_Plaster,HoardCat_Other,HoardCat_Copper,HoardCat_Total,AssMatCul_Qual,AssMatCul_Bone_Animal,AssMatCul_Bone_Human,AssMatCul_Bone_Objects,AssMatCul_Brooches,AssMatCul_CA_Vessel,AssMatCul_CA_Other,AssMatCul_Bronze_Objects,AssMatCul_Ceramic_Vessels,AssMatCul_Ceramic_Other,AssMatCul_PW_VesselsPlates,AssMatCul_SV_VesselsPlates,AssMatCul_SV_Other,AssMatCul_Gold,AssMatCul_Lead,AssMatCul_Iron,AssMatCul_uncertainMetal,AssMatCul_Wood_Vessels,AssMatCul_Wood_Other,AssMatCul_Glass,AssMatCul_ShaleJet,AssMatCul_Leather,AssMatCul_Stone,AssMatCul_Plantfibre,AssMatCul_Ivory,AssMatCul_Other,AssMatCul_SumCat,AssMatCul_VesselCount,AssMatCult_Notes,ID no.
0,57,AC_Excavated,IARCH-99712F,NaN,ROMAN,"Antonine Empress, uncertain, 138-185",NaN,NaN,NaN,NaN,NaN,138.0,180.0,NaN,"West Parade site Area III, Roman interval towe...",Hoard identified from report by AC - coin deta...,N.B. NGR based on site plan & Magic Map. Stee...,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,3,0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Yes,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3242
1,76,AC_Excavated,IARCH-145D5D,NaN,ROMAN,Arcadius,NaN,NaN,NaN,NaN,NaN,383.0,408.0,Date of latest ruler/issuer,…hoard of over 115 3rd- & 4th-century coins (G...,No details appear to be published. There is a ...,"N.B. Quite steep SE facing slope, part of a NE...",1.0,NaN,115.0,NaN,115.0,NaN,NaN,NaN,NaN,115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,632


In [ ]:
more_finds = setting_coin_finds(additional_hoards)
more_finds = setFindsGeo(additional_hoards, more_finds)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
more_finds[more_finds['endDate'] > 324]
finds[finds['endDate'] <= 324]
cleaned_more_finds = more_finds[more_finds['hoard_id'] > 0]
complete_finds = pd.concat([finds, cleaned_more_finds])
complete_finds #1441
complete_finds['unique_id'].value_counts() # this is apparently NOT unique (see below), but 'hoard_gis_id' IS unique
# IARCH-93F761 = 3215 [double entry, with 216 or 259 coins]. the online version has 216 coins; hoard_gis_id to remove: 352, INDEX = 28
# IARCH-24CAD8 = 2937 (also online), 977 (mistake online). hoard_gis_id to remove: 207, INDEX = 262
# IARCH-78C7F4 = 3164 [double entry, online has Hillingdon 3164]. hoard_gis_id to remove: 325, INDEX = 789
# IARCH-D61D2D = 485 (also online), 1536 (mistake online). hoard_gis_id: 394, INDEX = 434
# all the hoard_gis_ids above are unique, but INDEX is NOT
#complete_finds[complete_finds['hoard_gis_id'] == 352]
complete_finds = complete_finds[complete_finds['hoard_gis_id'].isin([352, 207, 325, 394]) == False]

# url is https://finds.org.uk/database/hoards/record/id/10 - the last number is to be replaced with the hoard_id. Unique ID and Name appear on the website. Not sure where hoard_gis_id appears.
# 5.2.21 - this is the merged list of both datasets (while also taking IDs from the online version). NOTE: It isn't filtered yet (e.g. end date should be > 324). 

In [ ]:
complete_finds.head(5)


,hoard_id,hoard_gis_id,unique_id,name,startDate,endDate,type_find,hoard?,excavation?,single?,num_coins,num_known_coins,num_unknown_coins,year_found,year_end_found,comments,lat,long,certainty,owner,created,imported
0,4.0,2083.0,IARCH-15228D,PAS: Newton Mills 4,383.0,388.0,EG_IARCH,hoard,NaN,NaN,255.0,255.0,0.0,1983.0,1983.0,"Robertson 2000, 358 no. 1474:""The hoard of 255...",51.3833,-2.4181,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105
1,10.0,193.0,IARCH-CB0ABA,PAS: Camerton 10,393.0,402.0,AC_Excavated,hoard,excav,NaN,26.0,26.0,0.0,1814.0,1814.0,"""At Camerton three Roman villas were discovere...",51.3041,-2.44894,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105
2,12.0,1553.0,IARCH-3BA394,PAS: Pewsey 12,395.0,402.0,EG_IARCH,hoard,NaN,NaN,32.0,32.0,0.0,2009.0,2012.0,2009 T233 (25 siliquae); 2010 T746 (2 siliquae...,51.3046,-1.73744,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105
3,14.0,248.0,IARCH-40793E,PAS: Catsgore 14,337.0,341.0,AC_Excavated,hoard,excav,NaN,28.0,28.0,0.0,1971.0,1972.0,E.M. Besly listed 28 AE coins (Catsgore hoard ...,51.0355,-2.70262,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105
4,15.0,1999.0,IARCH-4096EB,PAS: East Harptree 15,367.0,378.0,EG_IARCH,hoard,NaN,NaN,1496.0,1496.0,0.0,1887.0,1887.0,"Robertson 2000, 348 no. 1424:""""The spot [i.e. ...",51.2876,-2.64552,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105


13.4.21 - tasks:


*   Merge the two files for Coin Groups
*   Test that the Coin Finds & Groups are correctly associated (10 cases)
*   Test Coin Finds have all that appears in the csv (5 cases)
*   Test Coin Groups have all that appears in the csv (5 cases)
*   Scrape all data off website (first based on IDs in our side - for comparison) - can be downloaded as JSONs (https://finds.org.uk/database/hoards/record/id/65/format/json)
*   Compare what we have to the scraped data and make edits accordingly




In [ ]:
complete_finds.to_csv('complete_online_finds.csv')
files.download('complete_online_finds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<h2>Dealing with the coin groups</h2>

In [ ]:
brit_imported_coin_groups = pd.read_csv('Roman hoards content summaries_short.csv')
len(brit_imported_coin_groups) # 25826

for i in range(len(brit_imported_coin_finds)):
    if brit_imported_coin_finds.iloc[i].GIS_ID in all_hoard_numbers: # all the old-find-id values in the find list 
        all_hoard_numbers.discard(brit_imported_coin_finds.iloc[i].GIS_ID) # remove all the hoards already examined from the list
print(len(all_hoard_numbers)) # 662, these are hoards not examined previously.



for i in range(len(brit_imported_coin_groups)):
    if brit_imported_coin_groups.iloc[i].id in all_coin_groups_numbers: # all_coin_groups_numbers = set(coin_summary.id.values)
        all_coin_groups_numbers.discard(brit_imported_coin_groups.iloc[i].id)
print(len(all_coin_groups_numbers)) # 1148

additional_coin_finds = coin_summary[coin_summary['id'].isin(all_coin_groups_numbers)] # 1148 in length
additional_coin_finds.head()

#additional_hoards.to_csv('additional_hoards.csv')
#additional_coin_finds.to_csv('additional_coin_groups.csv')

brit_imported_coin_groups.head()

624
1148


,id,hoard,hoardID,broadperiod,ruler,denomination,fromDate,toDate,mint,geography,quantity,created,createdBy,updated,updatedBy
0,26878,BM-0D5E3C,3252,ROMAN,Tetrarchic Ruler (uncertain issuer),Nummus (AE 1 - AE 4),298.0,299.0,Trier,NaN,1.0,2015-10-16T11:49:38Z,26767,2015-10-16T11:55:42Z,26767.0
1,26879,BM-0D5E3C,3252,ROMAN,Maximian I,Nummus (AE 1 - AE 4),300.0,305.0,NaN,NaN,1.0,2015-10-16T11:56:12Z,26767,NaN,NaN
2,26880,BM-0D5E3C,3252,ROMAN,Tetrarchic Ruler (uncertain issuer),Nummus (AE 1 - AE 4),300.0,305.0,Lyon,NaN,1.0,2015-10-16T11:56:52Z,26767,NaN,NaN
3,19070,BM-B554B4,3020,ROMAN,House of Constantine,Nummus (AE 1 - AE 4),337.0,NaN,NaN,NaN,7.0,2015-02-11T13:14:54Z,528,NaN,NaN
4,19071,BM-B554B4,3020,ROMAN,Unspecified ruler (contemporary copy),Nummus (AE 1 - AE 4),NaN,NaN,NaN,NaN,3.0,2015-02-11T13:15:12Z,528,2015-02-11T13:16:04Z,528.0


In [ ]:

_ = files.upload()
coin_summary = pd.read_excel('CoinSummary.xlsx')

print(len(coin_summary)) # 26788, indices read 26868
all_coin_groups_numbers = set(coin_summary.id.values)
print(len(all_coin_groups_numbers)) # 26788 - meaning no duplicates here

Saving CoinSummary.xlsx to CoinSummary.xlsx
26788
26788


In [ ]:




# taking in the extra two files:
################################
additional_finds = pd.read_csv('additional_hoards.csv', encoding='ISO-8859-1')
additional_coin_groups = pd.read_csv('additional_coin_groups.csv', encoding='ISO-8859-1')

more_groups = setting_coin_groups(additional_coin_groups)

more_groups = initial_filtering(more_groups)
more_groups = coin_group_cleaning(more_groups)

more_groupsB = reg_update_coin_groups(2015, 6, 1, more_groups)
################################


############ merging both files ###########
groups = pd.concat([groups, more_groups])
groupB = pd.concat([groupB, more_groupsB])
finds = pd.concat([finds, more_finds])


# saving to files
groups.to_csv('coin_groups.csv')
groupB.to_csv('coin_groups_to_update.csv')	# this returns only those entries which require updating
finds.to_csv('coin_finds.csv')

#testing_database_connections()

'Unspecified ruler (contemporary copy)'

In [ ]:
demo_dict = {'Unspecified ruler (contemporary copy)':'bla bla'}
for i in range(3,8):
  print(groups.at[i,'ruler'] in demo_dict)

False
True
False
False
False


In [ ]:
groups['ruler'].head()

3                     House of Constantine
4    Unspecified ruler (contemporary copy)
5                     House of Valentinian
6                     House of Valentinian
7                     House of Valentinian
Name: ruler, dtype: object

In [ ]:
groups.index

Int64Index([    3,     4,     5,     6,     7,     8,     9,    28,    29,
               30,
            ...
            25772, 25773, 25774, 25775, 25776, 25777, 25778, 25779, 25780,
            25781],
           dtype='int64', length=7427)